### Vanilla CTR Zeroshot

In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'zeroshot_only_description_inference_valid.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

624


In [3]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [4]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 354 -- No: 41 -- Invalid: 229
invalid_users: ['0', '9', '11', '12', '17', '18', '19', '21', '22', '24', '27', '33', '34', '35', '37', '38', '39', '40', '43', '44', '46', '49', '53', '54', '57', '58', '61', '62', '65', '68', '69', '72', '75', '79', '83', '86', '88', '89', '92', '93', '97', '100', '103', '104', '107', '108', '109', '111', '112', '114', '116', '118', '120', '122', '124', '127', '131', '135', '137', '138', '142', '143', '153', '154', '158', '160', '161', '162', '164', '166', '171', '172', '173', '174', '175', '177', '182', '183', '184', '185', '188', '191', '192', '193', '194', '196', '198', '199', '207', '209', '210', '220', '221', '222', '223', '228', '229', '238', '239', '240', '243', '244', '247', '248', '253', '256', '260', '261', '265', '266', '272', '275', '279', '280', '282', '283', '285', '286', '287', '291', '293', '299', '304', '305', '306', '307', '310', '318', '319', '325', '326', '329', '330', '332', '333', '339', '347', '348', '349', '355', '360', '367'

In [6]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,6,0,5.0,1461801600
14,12,1,5.0,1504656000
21,12,2,4.0,1506643200
27,20,3,5.0,1493337600
33,21,4,5.0,1524614400


In [7]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 26  17]
 [220 353]]
AUC: 0.6103535046065182
Precision: 0.9540540540540541
Recall: 0.6160558464223386
Accuracy: 0.6152597402597403


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,5,0,5.0,1404604800
13,11,1,5.0,1504656000
20,11,2,4.0,1506643200
26,19,3,5.0,1493337600
32,19,4,5.0,1524614400


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 35  11]
 [235 343]]
AUC: 0.6771475853768618
Precision: 0.9689265536723164
Recall: 0.5934256055363322
Accuracy: 0.6057692307692307
